In [2]:
import json, os, subprocess
from ruamel.yaml import YAML
import pandas as pd
from dataclasses import dataclass, asdict, field
pd.options.display.max_columns = 100

In [3]:
@dataclass
class EnvironmentInfo:
    env_name: str
    env_path: str

In [4]:
def list_conda_environments() -> list[EnvironmentInfo]:
    # Run 'conda env list' to get a list of environments
    result = subprocess.run(['conda', 'env', 'list'], stdout=subprocess.PIPE, text=True)
    
    # Parse the result
    envs = []
    for line in result.stdout.splitlines():
        # Environments are listed with their paths
        if line and not line.startswith('#'):
            items = line.split(" ")
            if items[0]:
                env_name = items[0]
                env_path = items[-1]
                envs.append(EnvironmentInfo(env_name, env_path))    
    return envs

In [5]:
def list_env_packages(env_name) -> dict:
    # Run 'conda list' for the specific environment
    result = subprocess.run(['conda', 'list', '-n', env_name, '--json'], stdout=subprocess.PIPE, text=True)

    packages = json.loads(result.stdout)
    
    return {pkg['name']: pkg['version'] for pkg in packages}

In [6]:
envs = list_conda_environments()
envs

[EnvironmentInfo(env_name='base', env_path='C:\\Users\\holge\\anaconda3'),
 EnvironmentInfo(env_name='build_env', env_path='C:\\Users\\holge\\anaconda3\\envs\\build_env'),
 EnvironmentInfo(env_name='google', env_path='C:\\Users\\holge\\anaconda3\\envs\\google'),
 EnvironmentInfo(env_name='influx', env_path='C:\\Users\\holge\\anaconda3\\envs\\influx'),
 EnvironmentInfo(env_name='user_env', env_path='C:\\Users\\holge\\anaconda3\\envs\\user_env')]

In [7]:
env_records = []
for env in envs:
    env_record = asdict(env)
    env_record.update(list_env_packages(env.env_name))
    env_records.append(env_record)

In [8]:
df = pd.DataFrame.from_records(env_records)
df.tail()

,env_name,env_path,_anaconda_depends,abseil-cpp,aiobotocore,aiofiles,aiohttp,aioitertools,aiosignal,aiosqlite,alabaster,altair,anaconda-anon-usage,anaconda-catalogs,anaconda-client,anaconda-cloud-auth,anaconda-navigator,anaconda-project,analytics-python,anyio,aom,appdirs,archspec,argon2-cffi,argon2-cffi-bindings,arrow,arrow-cpp,astor,astroid,astropy,asttokens,async-timeout,atomicwrites,attrs,automat,autopep8,aws-c-common,aws-c-event-stream,aws-checksums,aws-sdk-cpp,babel,backcall,backoff,backports,backports.functools_lru_cache,backports.tempfile,backports.weakref,bcrypt,beautifulsoup4,binaryornot,...,aws-c-auth,aws-c-cal,aws-c-compression,aws-c-http,aws-c-io,aws-c-mqtt,aws-c-s3,aws-c-sdkutils,aws-crt-cpp,blinker,gettext,gitdb,gitpython,glib-tools,gst-plugins-base,gstreamer,influxdb,libabseil,libarrow,libarrow-acero,libarrow-dataset,libarrow-flight,libarrow-flight-sql,libarrow-gandiva,libarrow-substrait,libblas,libcblas,libcrc32c,libglib,libgoogle-cloud,libgoogle-cloud-storage,libgrpc,libhwloc,liblapack,libogg,libparquet,libre2-11,libutf8proc,libvorbis,pthreads-win32,pydantic-core,pydeck,rich,seaborn-base,smmap,streamlit,tomli,tzlocal,validators,trial_p
0,base,C:\Users\holge\anaconda3,2023.09,20211102.0,2.5.0,22.1.0,3.8.5,0.7.1,1.2.0,0.18.0,0.7.12,5.2.0,0.4.2,0.2.0,1.12.1,0.1.3,2.5.0,0.11.1,2.2.2,3.5.0,3.6.0,1.4.4,0.2.2,21.3.0,21.2.0,1.2.3,11.0.0,0.8.1,2.14.2,5.1,2.0.5,4.0.2,1.4.0,22.1.0,20.2.0,1.6.0,0.6.8,0.1.6,0.1.11,1.8.185,2.11.0,0.2.0,2.2.1,1.1,1.6.4,1.0,1.0.post1,3.2.0,4.12.2,0.4.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,build_env,C:\Users\holge\anaconda3\envs\build_env,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.12.1,NaN,NaN,0.11.1,NaN,NaN,NaN,NaN,0.2.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.12.2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,google,C:\Users\holge\anaconda3\envs\google,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.4.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,influx,C:\Users\holge\anaconda3\envs\influx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.8.1,NaN,NaN,2.4.1,NaN,NaN,23.2.0,NaN,NaN,0.9.14,0.4.2,0.1.18,1.11.267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.7.16,0.6.10,0.2.18,0.8.1,0.14.6,0.10.3,0.5.3,0.1.15,0.26.4,1.7.0,0.21.1,4.0.11,3.1.42,2.80.0,1.22.9,1.22.9,5.3.1,20240116.1,15.0.2,15.0.2,15.0.2,15.0.2,15.0.2,15.0.2,15.0.2,3.9.0,3.9.0,1.1.2,2.80.0,2.22.0,2.22.0,1.62.1,2.9.3,3.9.0,1.3.4,15.0.2,2023.09.01,2.8.0,1.3.7,2.9.1,2.16.3,0.8.0,13.7.1,0.13.2,5.0.0,1.32.2,2.0.1,5.2,0.24.0,NaN
4,user_env,C:\Users\holge\anaconda3\envs\user_env,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023.1


## Only show desired columns

In [9]:
with open("config.yaml", "r") as file:
    config = YAML().load(file)
    
cols = config["packages"]
cols = set(cols).intersection(set(df.columns))
cols = ['env_name', 'env_path'] + list(cols)
df = df[cols]

In [10]:
df = df.set_index("env_name")
df.fillna("")

,env_path,conda,matplotlib,pandas,python,plotly,openpyxl,flask,pytest,ipykernel,ruamel.yaml,pip,pydantic,numpy
env_name,,,,,,,,,,,,,,
base,C:\Users\holge\anaconda3,23.11.0,3.7.2,2.0.3,3.11.5,5.9.0,3.0.10,2.2.2,7.4.0,6.25.0,0.17.21,23.2.1,1.10.8,1.24.3
build_env,C:\Users\holge\anaconda3\envs\build_env,23.11.0,,,3.11.7,,,,,,0.18.5,23.3.1,,
google,C:\Users\holge\anaconda3\envs\google,24.9.1,,,3.11.10,,,,,6.29.5,0.18.6,24.2,,
influx,C:\Users\holge\anaconda3\envs\influx,,3.8.3,2.2.1,3.12.2,5.21.0,3.1.2,,,6.29.3,0.18.6,24.0,2.6.3,1.26.4
user_env,C:\Users\holge\anaconda3\envs\user_env,,,,3.12.0,,,,,,,23.3.2,,


In [16]:
s = df.fillna("").to_markdown()
print(s+"\n")

| env_name   | env_path                                | conda   | matplotlib   | pandas   | python   | plotly   | openpyxl   | flask   | pytest   | ipykernel   | ruamel.yaml   | pip    | pydantic   | numpy   |
|:-----------|:----------------------------------------|:--------|:-------------|:---------|:---------|:---------|:-----------|:--------|:---------|:------------|:--------------|:-------|:-----------|:--------|
| base       | C:\Users\holge\anaconda3                | 23.11.0 | 3.7.2        | 2.0.3    | 3.11.5   | 5.9.0    | 3.0.10     | 2.2.2   | 7.4.0    | 6.25.0      | 0.17.21       | 23.2.1 | 1.10.8     | 1.24.3  |
| build_env  | C:\Users\holge\anaconda3\envs\build_env | 23.11.0 |              |          | 3.11.7   |          |            |         |          |             | 0.18.5        | 23.3.1 |            |         |
| google     | C:\Users\holge\anaconda3\envs\google    | 24.9.1  |              |          | 3.11.10  |          |            |         |          | 6.29.5 

In [11]:
df.to_excel("envs_table.xlsx")